# Key Actions

> Declarative keyboard action bindings supporting HTMX triggers and JS callbacks.

In [ ]:
#| default_exp core.actions

In [ ]:
#| export
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Optional

from cjm_fasthtml_keyboard_navigation.core.key_mapping import format_key_combo

## KeyAction

Declares a keyboard shortcut and its associated action. Actions can trigger HTMX requests, call JS functions, or switch modes.

In [ ]:
#| export
@dataclass
class KeyAction:
    """A keyboard shortcut binding."""

    # Trigger
    key: str  # JavaScript key name (e.g., "Enter", " ", "ArrowUp")
    modifiers: frozenset[str] = field(
        default_factory=frozenset
    )  # required modifiers ("shift", "ctrl", "alt", "meta")

    # Action - exactly one should typically be set
    htmx_trigger: Optional[str] = None  # ID of hidden button to click
    js_callback: Optional[str] = None  # JS function name to call
    mode_enter: Optional[str] = None  # mode name to enter
    mode_exit: bool = False  # exit current mode (return to default)

    # Behavior
    prevent_default: bool = True  # call e.preventDefault()
    stop_propagation: bool = False  # call e.stopPropagation()

    # Conditions
    zone_ids: Optional[tuple[str, ...]] = None  # only in these zones (None = all)
    mode_names: Optional[tuple[str, ...]] = None  # only in these modes (None = all)
    not_modes: Optional[tuple[str, ...]] = None  # not in these modes
    custom_condition: Optional[str] = None  # raw JS expression for additional conditions

    # Documentation
    description: str = ""  # human-readable description for hints
    hint_group: str = "General"  # grouping for keyboard hints display
    show_in_hints: bool = True  # whether to show in keyboard hints

    def matches_context(
        self,
        zone_id: str,  # current active zone
        mode_name: str  # current mode
    ) -> bool:         # True if action is valid in this context
        """Check if action is valid for given zone and mode."""
        # Check zone condition
        if self.zone_ids is not None and zone_id not in self.zone_ids:
            return False
        
        # Check mode condition
        if self.mode_names is not None and mode_name not in self.mode_names:
            return False
        
        # Check not_modes condition
        if self.not_modes is not None and mode_name in self.not_modes:
            return False
        
        return True

    def get_display_key(self) -> str: # formatted key combo for display
        """Get formatted key combination for display."""
        return format_key_combo(self.key, self.modifiers)

    def to_js_config(self) -> dict: # JavaScript-compatible configuration
        """Convert to JavaScript configuration object."""
        return {
            "key": self.key,
            "modifiers": list(self.modifiers),
            "htmxTrigger": self.htmx_trigger,
            "jsCallback": self.js_callback,
            "modeEnter": self.mode_enter,
            "modeExit": self.mode_exit,
            "preventDefault": self.prevent_default,
            "stopPropagation": self.stop_propagation,
            "zoneIds": list(self.zone_ids) if self.zone_ids else None,
            "modeNames": list(self.mode_names) if self.mode_names else None,
            "notModes": list(self.not_modes) if self.not_modes else None,
            "customCondition": self.custom_condition,
            "description": self.description,
            "hintGroup": self.hint_group,
        }

In [ ]:
# Test basic KeyAction
action = KeyAction(
    key=" ",  # Space
    htmx_trigger="toggle-btn",
    description="Toggle selection",
    hint_group="Selection"
)

assert action.key == " "
assert action.htmx_trigger == "toggle-btn"
assert action.get_display_key() == "Space"
assert action.matches_context("any-zone", "any-mode") == True

In [ ]:
# Test action with modifiers
shift_action = KeyAction(
    key="ArrowUp",
    modifiers=frozenset({"shift"}),
    htmx_trigger="reorder-up",
    zone_ids=("queue",),
    description="Move item up"
)

assert shift_action.get_display_key() == "Shift+↑"
assert shift_action.matches_context("queue", "navigation") == True
assert shift_action.matches_context("browser", "navigation") == False

In [ ]:
# Test mode-specific action
split_action = KeyAction(
    key="Enter",
    htmx_trigger="execute-split",
    mode_names=("split",),
    description="Split at caret"
)

assert split_action.matches_context("any", "split") == True
assert split_action.matches_context("any", "navigation") == False

In [ ]:
# Test action with not_modes
nav_only_action = KeyAction(
    key="Enter",
    mode_enter="split",
    not_modes=("split",),  # don't enter split if already in split
    description="Enter split mode"
)

assert nav_only_action.matches_context("zone", "navigation") == True
assert nav_only_action.matches_context("zone", "split") == False

In [ ]:
# Test JS callback action
audition_action = KeyAction(
    key="ArrowDown",
    js_callback="auditionCurrent",
    zone_ids=("vad-timeline",),
    description="Navigate and audition"
)

config = audition_action.to_js_config()
assert config["jsCallback"] == "auditionCurrent"
assert config["htmxTrigger"] is None
assert config["zoneIds"] == ["vad-timeline"]

## Common Action Patterns

In [ ]:
# Example: Toggle selection (common for browser/list UIs)
toggle_space = KeyAction(
    key=" ",
    htmx_trigger="toggle-btn",
    description="Toggle selection",
    hint_group="Selection"
)

toggle_enter = KeyAction(
    key="Enter",
    htmx_trigger="toggle-btn",
    not_modes=("split", "edit"),  # don't toggle in edit modes
    description="Toggle selection",
    hint_group="Selection",
    show_in_hints=False  # don't duplicate in hints since Space is shown
)

# Example: Delete/Remove actions
delete_action = KeyAction(
    key="Delete",
    htmx_trigger="delete-btn",
    description="Delete item",
    hint_group="Actions"
)

backspace_delete = KeyAction(
    key="Backspace",
    htmx_trigger="delete-btn",
    description="Delete item",
    hint_group="Actions",
    show_in_hints=False  # alternative key
)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()